In [2]:
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
import tensorflow as tf

In [4]:
with tf.device('/cpu:0'):
    input_tensor = Input((299, 299, 3))
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(1)(p_out)
    predictions = Dense(16, activation='softmax')(p_out)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.load_weights('Xception_finetune_40.h5')

parallel_model = multi_gpu_model(model, gpus=2)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 299, 299, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_4[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

In [27]:
batch_size = 128
img_size = (299, 299)

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("/home/super-speed-data/train-data-xception/20181019/test-ready/", 
                                         img_size, 
                                         shuffle=False,
                                         batch_size=batch_size)
test_img_nums = test_generator.samples
all_test_results = parallel_model.predict_generator(test_generator, (test_img_nums//batch_size) + 1, verbose=1)
all_test_results = all_test_results[:test_img_nums]
all_labels = test_generator.classes

Found 6131 images belonging to 16 classes.
48/48 [==============================] - 22s 448ms/step


In [67]:
class_label_dict = test_generator.class_indices

def get_key(dict, value):
    return [k for k, v in dict.items() if v == value]

# create class num lens dict, every dict store current class predict num
total_predictions_dict = {}
for row_class_name, label in class_label_dict.items():
    total_predictions_dict[row_class_name] = {}
    for column_class_name, label in class_label_dict.items():
        total_predictions_dict[row_class_name][column_class_name] = 0

In [68]:
for i, label in enumerate(all_labels):
    # get the first result
    label_class_name = get_key(class_label_dict, label)[0]
    test_class_name = get_key(class_label_dict, np.argmax(test[i]))[0]
#     print(label_class_name)
#     print(total_predictions_dict[label_class_name].keys())
    total_predictions_dict[label_class_name][test_class_name] += 1

In [69]:
print(total_predictions_dict['RC'])

{'RC': 41, 'LSIL': 1, 'TRI': 0, 'EC': 0, 'GEC': 9, 'ASCUS': 0, 'HSIL': 6, 'AGC': 0, 'ACTINO': 0, 'SC': 2, 'MC': 0, 'VIRUS': 0, 'FUNGI': 0, 'ASCH': 0, 'SCC': 0, 'CC': 0}


In [64]:
print(total_predictions_dict['LSIL'])

{'RC': 0, 'LSIL': 603, 'TRI': 0, 'EC': 0, 'GEC': 0, 'ASCUS': 24, 'HSIL': 2, 'AGC': 0, 'ACTINO': 0, 'SC': 0, 'MC': 3, 'VIRUS': 0, 'FUNGI': 0, 'ASCH': 0, 'SCC': 2, 'CC': 1}


In [66]:
print(total_predictions_dict['HSIL'])

{'RC': 2, 'LSIL': 4, 'TRI': 0, 'EC': 0, 'GEC': 13, 'ASCUS': 0, 'HSIL': 1642, 'AGC': 1, 'ACTINO': 0, 'SC': 20, 'MC': 0, 'VIRUS': 1, 'FUNGI': 0, 'ASCH': 1, 'SCC': 8, 'CC': 0}


In [90]:
import csv
out = open('confusion_matrix.csv','a', newline='')
csv_write = csv.writer(out,dialect='excel')

# write the title
line = [class_name for class_name, label in class_label_dict.items()]
line = [" "] + line + ["TOTAL"]
csv_write.writerow(line)

# write rows

for row_class_name, label in class_label_dict.items():
    one_class_total_predict = 0
    line = [row_class_name]
    for column_class_name, label in class_label_dict.items():
        one_class_total_predict += total_predictions_dict[row_class_name][column_class_name]
    
    for column_class_name, label in class_label_dict.items():
        one_class_cur_predict = total_predictions_dict[row_class_name][column_class_name]
        # acc
        #acc = round((one_class_cur_predict / one_class_total_predict), 4)
        #line.append(str(acc))
        # num
        line.append(one_class_cur_predict)
    print(one_class_total_predict)
    print(str(one_class_total_predict))
    line.append(str(one_class_total_predict))
    print(line)
    csv_write.writerow(line)
out.close()

59
59
['RC', 41, 1, 0, 0, 0, 0, 6, 0, 2, 0, 0, 0, 9, 0, 0, 0, '59']
635
635
['LSIL', 0, 603, 0, 0, 0, 24, 2, 0, 0, 3, 0, 0, 0, 0, 2, 1, '635']
31
31
['ASCH', 2, 0, 15, 0, 0, 0, 9, 3, 2, 0, 0, 0, 0, 0, 0, 0, '31']
15
15
['EC', 1, 0, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, '15']
676
676
['TRI', 0, 0, 0, 0, 674, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, '676']
135
135
['ASCUS', 0, 32, 0, 0, 0, 96, 0, 0, 1, 5, 0, 0, 1, 0, 0, 0, '135']
1692
1692
['HSIL', 2, 4, 1, 0, 0, 0, 1642, 1, 20, 0, 0, 1, 13, 0, 8, 0, '1692']
81
81
['AGC', 0, 0, 0, 0, 0, 0, 0, 76, 0, 0, 0, 0, 4, 0, 1, 0, '81']
264
264
['SC', 6, 0, 0, 0, 0, 1, 18, 0, 217, 8, 0, 1, 13, 0, 0, 0, '264']
244
244
['MC', 0, 1, 0, 0, 0, 2, 0, 0, 1, 238, 0, 0, 0, 0, 0, 2, '244']
46
46
['FUNGI', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 44, 0, 0, 0, 0, 1, '46']
170
170
['VIRUS', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 170, 0, 0, 0, 0, '170']
248
248
['GEC', 9, 0, 1, 0, 0, 0, 5, 1, 10, 3, 0, 0, 219, 0, 0, 0, '248']
467
467
['ACTINO', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3